# Data Modelling
### Tried beginner level Data Modelling using Store Retail Dataset from Kaggle

In [3]:
import psycopg2
import pandas as pd

In [4]:
stores_data = pd.read_csv("Datasets/stores data-set.csv")

In [5]:
stores_data.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [6]:
sales_data = pd.read_csv("Datasets/sales data-set.csv")

In [7]:
sales_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [8]:
features_data = pd.read_csv("Datasets/Features data set.csv")

In [9]:
features_data.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [11]:
features_data_clean = features_data[['Store','Date','Temperature','Fuel_Price','CPI','Unemployment','IsHoliday']]

In [12]:
features_data_clean.head()

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,211.350143,8.106,False


In [43]:
def create_database():
    # connect to database
    try:
        conn=psycopg2.connect("host=localhost dbname=postgres user=postgres password=passw0rd")
        conn.set_session(autocommit=True)
        cur=conn.cursor()
        #create new database called Retail
        cur.execute("DROP DATABASE IF EXISTS retail")
        cur.execute("CREATE DATABASE retail")
        
        #close existing connection
        conn.close()
        
        #connect to Retail database
        conn=psycopg2.connect("host=localhost dbname=retail user=postgres password=passw0rd")
        conn.set_session(autocommit=True)
        cur=conn.cursor()
        
        
    except psycopg2.Error as e:
        print("There was error connecting to db")
        print(e)
        
    return cur, conn

In [45]:
cur, conn = create_database()

In [47]:
stores_table_create = ("""CREATE TABLE IF NOT EXISTS stores(
Store int PRIMARY KEY, Type VARCHAR, Size int)""")

In [17]:
sales_table_create = ("""CREATE TABLE IF NOT EXISTS sales(
Store int, Dept int, Date TIMESTAMP, Weekly_Sales decimal, IsHoliday Varchar)""")

In [51]:
features_table_create = ("""CREATE TABLE IF NOT EXISTS features(
Store int, Date TIMESTAMP, Temperature decimal, Fuel_Price decimal, CPI decimal, Unemployment decimal, IsHoliday Varchar)""")

In [48]:
cur.execute(stores_table_create)

In [49]:
cur.execute(sales_table_create)

In [52]:
cur.execute(features_table_create)

In [53]:
stores_table_insert = ("""INSERT INTO stores(Store, Type,Size) values(%s,%s,%s)""")

In [55]:
for i,row in stores_data.iterrows():
    cur.execute(stores_table_insert, list(row))

In [58]:
sales_table_insert = ("""INSERT INTO sales(Store,Dept,Date,Weekly_Sales,IsHoliday) values(%s,%s,%s,%s,%s)""")

In [59]:
for i,row in sales_data.iterrows():
    cur.execute(sales_table_insert, list(row))

In [60]:
features_table_insert = ("""INSERT INTO features
(Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday) values(%s,%s,%s,%s,%s,%s,%s)""")

In [61]:
for i,row in features_data_clean.iterrows():
    cur.execute(features_table_insert, list(row))